In [2]:
pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import wooldridge as woo
import pandas as pd
import linearmodels as plm

## Has the Return to Education changed over Time?

In [2]:
wagepan = woo.dataWoo('wagepan')
wagepan.head(20)

,nr,year,agric,black,bus,construc,ent,exper,fin,hisp,...,union,lwage,d81,d82,d83,d84,d85,d86,d87,expersq
0,13,1980,0,0,1,0,0,1,0,0,...,0,1.197540,0,0,0,0,0,0,0,1
1,13,1981,0,0,0,0,0,2,0,0,...,1,1.853060,1,0,0,0,0,0,0,4
2,13,1982,0,0,1,0,0,3,0,0,...,0,1.344462,0,1,0,0,0,0,0,9
3,13,1983,0,0,1,0,0,4,0,0,...,0,1.433213,0,0,1,0,0,0,0,16
4,13,1984,0,0,0,0,0,5,0,0,...,0,1.568125,0,0,0,1,0,0,0,25
5,13,1985,0,0,1,0,0,6,0,0,...,0,1.699891,0,0,0,0,1,0,0,36
6,13,1986,0,0,1,0,0,7,0,0,...,0,-0.720263,0,0,0,0,0,1,0,49
7,13,1987,0,0,1,0,0,8,0,0,...,0,1.669188,0,0,0,0,0,0,1,64
8,17,1980,0,0,0,0,0,4,0,0,...,0,1.675962,0,0,0,0,0,0,0,16
9,17,1981,0,0,0,0,0,5,0,0,...,0,1.518398,1,0,0,0,0,0,0,25


In [3]:
wagepan = wagepan.set_index(['nr', 'year'], drop=False)

In [4]:
wagepan.columns

Index(['nr', 'year', 'agric', 'black', 'bus', 'construc', 'ent', 'exper',
       'fin', 'hisp', 'poorhlth', 'hours', 'manuf', 'married', 'min',
       'nrthcen', 'nrtheast', 'occ1', 'occ2', 'occ3', 'occ4', 'occ5', 'occ6',
       'occ7', 'occ8', 'occ9', 'per', 'pro', 'pub', 'rur', 'south', 'educ',
       'tra', 'trad', 'union', 'lwage', 'd81', 'd82', 'd83', 'd84', 'd85',
       'd86', 'd87', 'expersq'],
      dtype='object')

In [5]:
wagepan.educ[:20]

nr  year
13  1980    14
    1981    14
    1982    14
    1983    14
    1984    14
    1985    14
    1986    14
    1987    14
17  1980    13
    1981    13
    1982    13
    1983    13
    1984    13
    1985    13
    1986    13
    1987    13
18  1980    12
    1981    12
    1982    12
    1983    12
Name: educ, dtype: int64

In [6]:
# FE model estimation:

reg = plm.PanelOLS.from_formula(
formula='lwage ~ married + union + C(year)*educ + EntityEffects',
data=wagepan, drop_absorbed=True)
results = reg.fit()

/var/folders/3x/x3jgd5qx5_b2g84w2_jrj0_m0000gn/T/ipykernel_3059/2794028872.py:6: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

educ

  results = reg.fit()


In [7]:
# print regression table:
table = pd.DataFrame({'b': round(results.params, 4),
                      'se': round(results.std_errors, 4),
                      't': round(results.tstats, 4),
                      'pval': round(results.pvalues, 4)})

print(f'table: \n{table}\n')

table: 
                           b      se        t    pval
married               0.0548  0.0184   2.9773  0.0029
union                 0.0830  0.0194   4.2671  0.0000
C(year)[T.1980]       1.3625  0.0162  83.9031  0.0000
C(year)[T.1981]       1.3400  0.1452   9.2307  0.0000
C(year)[T.1982]       1.3567  0.1451   9.3481  0.0000
C(year)[T.1983]       1.3729  0.1452   9.4561  0.0000
C(year)[T.1984]       1.4468  0.1452   9.9617  0.0000
C(year)[T.1985]       1.4122  0.1451   9.7315  0.0000
C(year)[T.1986]       1.4281  0.1451   9.8404  0.0000
C(year)[T.1987]       1.4529  0.1452  10.0061  0.0000
C(year)[T.1981]:educ  0.0116  0.0123   0.9448  0.3448
C(year)[T.1982]:educ  0.0148  0.0123   1.2061  0.2279
C(year)[T.1983]:educ  0.0171  0.0123   1.3959  0.1628
C(year)[T.1984]:educ  0.0166  0.0123   1.3521  0.1764
C(year)[T.1985]:educ  0.0237  0.0123   1.9316  0.0535
C(year)[T.1986]:educ  0.0274  0.0123   2.2334  0.0256
C(year)[T.1987]:educ  0.0304  0.0123   2.4798  0.0132

